# 순환 신경망

> 3.2.4 장에 해당하는 코드

In [1]:
# 코드 3-35

import torch
import torch.nn as nn

class CustomRNN(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=True):
        super(CustomRNN, self).__init__()
        # 선형결합에 사용할 학습가능한 매개변수를 생성한다.
        self.weight_xh, self.weight_hh, self.bias = \
            self.init_weight(input_size, hidden_size)
        # 필요한 정보를 저장한다
        self.hidden_size = hidden_size
        self.batch_first = batch_first
        
    def forward(self, inputs):
        """
        rnn_cell 을 구동하기 위해 inputs 의 크기 (T, B, E) 형태가 되어야 한다.
         - T: 시퀀스 총 길이
         - B: 미니배치크기
         - E: 입력층 크기 
        """
        if self.batch_first:
            # 첫번째 차원이 미니배치 크기인 경우 전치연산으로 바꿔준다.
            inputs = inputs.transpose(0, 1)
        seqlen, batch_size, _ = inputs.size()
        # 0 time-step 에서 은닉층 값을 0으로 초기화 시킨다
        hidden = self.init_hidden(batch_size, self.hidden_size)
        # output에 은닉층의 출력값을 저장한다.
        output = []
        # 시퀀스의 총 길이만큼 순방향전파를 진행한다.
        for i in range(seqlen):
            hidden = self.rnn_cell(inputs[i], hidden)
            output.append(hidden)
        output = torch.stack(output)
        if self.batch_first:
            output = output.transpose(0, 1)
        # 모든 타임스텝의 은닉층 출력값과 마지막 타임 스텝의 은닉층 출력값을 각각 반환한다.
        return output, hidden
    
    def rnn_cell(self, x, h):
        """RNN Cell"""
        h = x.mm(self.weight_xh.t()) + h.mm(self.weight_hh.t()) + self.bias
        return torch.tanh(h)
    
    def init_hidden(self, batch_size, hidden_size):
        """0 타임스텝에서 은닉층의 초기화"""
        return torch.zeros(batch_size, hidden_size)
    
    def init_weight(self, input_size, hidden_size):
        """rnn_cell 의 선형결합을 위한 초기값"""
        weight_xh = torch.randn(hidden_size, input_size).requires_grad_()
        weight_hh = torch.randn(hidden_size, hidden_size).requires_grad_()
        bias = torch.zeros(1, hidden_size).requires_grad_()
        return weight_xh, weight_hh, bias

In [2]:
# 코드 3-36

# RNN, LSTM, GRU 호출 방법

rnn_layer = nn.RNN(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)
rnn_layer = nn.LSTM(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)
rnn_layer = nn.GRU(input_size=5, hidden_size=10, batch_first=True)
print(rnn_layer)

RNN(5, 10, batch_first=True)
LSTM(5, 10, batch_first=True)
GRU(5, 10, batch_first=True)


In [4]:
# 코드 3-37

import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(70)

## 노트북 14-word embedding 에서 사용한 예시 사용
sentence = "We are going to watch Avengers End Game".split()
vocab = {tkn: i for i, tkn in enumerate(sentence, 1)}  # 단어장 생성
vocab['<unk>'] = 0
# 수치화된 데이터를 단어로 바꾸기 위한 사전
rev_vocab = {v: k for k, v in vocab.items()}
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [rev_vocab.get(x) for x in y]  

def construct_data(sentence, vocab):
    """
    (input, target) 쌍으로 데이터를 생성한다.
    - 최종 형태(수치화된 단어 쌍):
     [(We,are) ,(are,going), (going,to), (to,watch), 
        (watch,Avengers), (Avengers,End), (End,Game)]
    """
    numericalize = lambda x: vocab.get(x) if vocab.get(x) is not None else 0
    totensor = lambda x: torch.LongTensor(x)
    idxes = [numericalize(token) for token in sentence]
    x, t = idxes[:-1], idxes[1:]
    return totensor(x).unsqueeze(0), totensor(t).unsqueeze(0)

class Net(nn.Module):
    """예제 문장을 출력하는 모델"""
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        """
        vocab_size: 단어장의 크기
        input_size: 임베딩 크기 = RNN의 입력층 크기
        hidden_size: RNN의 은닉층 크기
        """
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, 
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, 
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        """
        텐서 크기 변화 위한 문자 설명 
         - V: 단어장 크기
         - T: 시퀀스 총 길이
         - B: 미니배치 크기
         - E: 임베딩 크기 = RNN의 입력층 크기 
         - D: RNN 은닉층 크기
        """
        # 1. embedding 층을 총과해서 분산 표상 방식으로 단어를 표현
        # 크기변화: (B, T) > (B, T, D)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (B, T, D) > output (B, T, D), hidden (1, B, D)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (B, T, D) > (B, T, V)
        output = self.linear(output)
        # 4. 모델 출력: 
        # 크기변화: (B, T, V) > (B*T, V)
        return output.view(-1, output.size(2))
        
# -----------------------------------------
# 데이터 생성
x, t = construct_data(sentence, vocab)

# 모델 생성을 위한 하이퍼 파라미터 설정
vocab_size = len(vocab)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다
input_size = 5  # 임베딩된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss()
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

# 훈련 시작
for step in range(151):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(x)
    # 손실값 계산
    loss = loss_function(output, t.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 30 == 0:
        print("[{:02d}/151] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["We"] + decode(pred)))
        print()

[01/151] 2.3034 
We Avengers watch are going watch watch Avengers

[31/151] 1.9172 
We are Avengers going are Avengers End Game

[61/151] 1.5057 
We are going going watch Avengers End Game

[91/151] 1.0342 
We are going to watch Avengers End Game

[121/151] 0.6438 
We are going to watch Avengers End Game

[151/151] 0.3932 
We are going to watch Avengers End Game

